In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [4]:
from src.inference import get_feature_store

In [5]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-03 01:04:12,667 INFO: Initializing external client
2025-03-03 01:04:12,670 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 01:04:13,572 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215649
Fetching data from 2025-02-02 06:04:12.660996+00:00 to 2025-03-03 05:04:12.660996+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.33s) 


In [6]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

pickup_hour  pickup_location_id  rides
0      2025-02-02 07:00:00+00:00                   2      0
1      2025-02-02 08:00:00+00:00                   2      0
2      2025-02-02 09:00:00+00:00                   2      0
3      2025-02-02 10:00:00+00:00                   2      0
4      2025-02-02 11:00:00+00:00                   2      0
...                          ...                 ...    ...
174189 2025-03-03 00:00:00+00:00                 263     12
174190 2025-03-03 01:00:00+00:00                 263     13
174191 2025-03-03 02:00:00+00:00                 263      3
174192 2025-03-03 03:00:00+00:00                 263      4
174193 2025-03-03 04:00:00+00:00                 263      9

[174194 rows x 3 columns]

In [7]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174194 entries, 0 to 175699
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174194 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174194 non-null  int32                  
 2   rides               174194 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [8]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [9]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174194 entries, 0 to 175699
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174194 non-null  datetime64[us]
 1   pickup_location_id  174194 non-null  int32         
 2   rides               174194 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [10]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [11]:
features

rides_t-672  rides_t-671  rides_t-670  rides_t-669  rides_t-668  \
0              2            0            0            0            3   
1              2            1            0            1            1   
2            369          273          249           21          251   
3              0            0            0            0            0   
4              0            0            0            0            0   
..           ...          ...          ...          ...          ...   
246            0            0            0            0            0   
247            0            0            0            0            0   
248            0            2            0            0            4   
249            0            1            1            1            1   
250           11          287          206          131          101   

     rides_t-667  rides_t-666  rides_t-665  rides_t-664  rides_t-663  ...  \
0              0            0            3            5            1  ...   
1              1            0            1            1            1  ...   
2            348           17          373          286          251  ...   
3              0            0            0            0            0  ...   
4              0            1            0            0            0  ...   
..           ...          ...          ...          ...          ...  ...   
246            0            0            0            0            0  ...   
247            0            0            0            0            0  ...   
248            0            1            0            0            2  ...   
249            2            0            3            4            0  ...   
250          199          173           84           61          187  ...   

     rides_t-8  rides_t-7  rides_t-6  rides_t-5  rides_t-4  rides_t-3  \
0            0          2          0          0         16          0   
1            1          0          0          2          0          0   
2           32        393        166         35         89        335   
3            0          0          0          0          0          0   
4            1          0          1          1          0          0   
..         ...        ...        ...        ...        ...        ...   
246          0          0          0          0          0          0   
247          0          0          0          0          0          0   
248          0          0          2          0          0          0   
249          4          1          0          0          0          1   
250        124         77        172        251        262         13   

     rides_t-2  rides_t-1  pickup_location_id         pickup_hour  
0            0          3                 255 2025-02-03 14:00:00  
1            0          0                 218 2025-02-04 13:00:00  
2          118          9                 237 2025-02-26 16:00:00  
3            0          0                  57 2025-02-26 00:00:00  
4            0          0                  56 2025-02-15 01:00:00  
..         ...        ...                 ...                 ...  
246          0          0                 187 2025-02-07 23:00:00  
247          0          0                   8 2025-02-27 12:00:00  
248          2          0                  71 2025-02-15 11:00:00  
249          1          3                 168 2025-02-21 23:00:00  
250        147        147                 186 2025-02-13 02:00:00  

[251 rows x 674 columns]

In [12]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-03 01:04:28,679 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 01:04:28,690 INFO: Initializing external client
2025-03-03 01:04:28,691 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 01:04:29,305 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215649
Fetching data from 2025-02-02 06:04:28.678652+00:00 to 2025-03-03 05:04:28.678652+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (7.71s) 


In [13]:
current_date

Timestamp('2025-03-03 06:04:28.678652+0000', tz='Etc/UTC')

In [14]:
features

rides_t-672  rides_t-671  rides_t-670  rides_t-669  rides_t-668  \
0              0            0            0            0            0   
1              0            1            0            0            0   
2              2            2            3            0            4   
3              0            0            0            0            0   
4              1            1            6            4            3   
..           ...          ...          ...          ...          ...   
246            0            0            1            0            0   
247            4            2            1            1            2   
248            1            9           14            9           30   
249           30           44           59           85          107   
250           26           43           80          114          133   

     rides_t-667  rides_t-666  rides_t-665  rides_t-664  rides_t-663  ...  \
0              0            0            0            0            0  ...   
1              0            0            0            0            0  ...   
2              3            2            5            0            7  ...   
3              0            0            1            0            0  ...   
4              1            3            1            6            2  ...   
..           ...          ...          ...          ...          ...  ...   
246            0            0            0            0            0  ...   
247            0            3            1            0            0  ...   
248           28           30           28           38           26  ...   
249           91           91           76           75           59  ...   
250          156          127          128          132           94  ...   

     rides_t-8  rides_t-7  rides_t-6  rides_t-5  rides_t-4  rides_t-3  \
0            0          0          0          0          0          0   
1            0          0          0          0          0          0   
2           50         68         62         65         43         15   
3            0          0          0          0          0          0   
4            2          3          1          1          3          3   
..         ...        ...        ...        ...        ...        ...   
246          0          0          0          0          0          0   
247          1          0          0          0          0          0   
248         14          7         12          7         10          2   
249         18          7          5          5          2          5   
250         90         82         49         41         29         17   

     rides_t-2  rides_t-1  pickup_location_id         pickup_hour  
0            0          0                   2 2025-03-02 07:00:00  
1            0          0                   3 2025-03-02 07:00:00  
2            0          1                   4 2025-03-02 07:00:00  
3            0          0                   6 2025-03-02 07:00:00  
4            2          1                   7 2025-03-02 07:00:00  
..         ...        ...                 ...                 ...  
246          0          0                 259 2025-03-02 07:00:00  
247          0          2                 260 2025-03-02 07:00:00  
248          1          2                 261 2025-03-02 07:00:00  
249          8         10                 262 2025-03-02 07:00:00  
250          9         30                 263 2025-03-02 07:00:00  

[251 rows x 674 columns]

In [15]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-03 01:04:44,097 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 01:04:44,104 INFO: Initializing external client
2025-03-03 01:04:44,104 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 01:04:44,663 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215649


In [16]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [17]:
predictions

pickup_location_id  predicted_demand
0                     2               0.0
1                     3               0.0
2                     4               1.0
3                     6               0.0
4                     7               1.0
..                  ...               ...
246                 259               0.0
247                 260               1.0
248                 261               3.0
249                 262              28.0
250                 263              36.0

[251 rows x 2 columns]

In [18]:
predictions.sort_values("predicted_demand", ascending=False).head(10)

pickup_location_id  predicted_demand
122                 132             131.0
223                 236              71.0
217                 230              50.0
226                 239              47.0
174                 186              45.0
225                 238              41.0
151                 162              39.0
44                   48              38.0
224                 237              37.0
250                 263              36.0